## 使用 Anthropic API 的工具调用生成结构化输出

Anthropic API 最近添加了工具调用功能。

这对生成结构化输出非常有用。

In [ ]:
! pip install -U langchain-anthropic

In [ ]:
# 可选配置
import os
# os.environ['LANGSMITH_TRACING'] = 'true' # 启用追踪
# os.environ['LANGSMITH_API_KEY'] = <你的API密钥>

## 如何使用工具来生成结构化输出？

函数调用/工具使用会生成一个有效负载。

有效负载通常是一个 JSON 字符串，可以传递给 API 或者在本例中传递给解析器以生成结构化输出。

LangChain 提供了 `llm.with_structured_output(schema)` 方法，使得生成匹配 `schema` 的结构化输出变得非常容易。

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


# 数据模型
class code(BaseModel):
    """代码输出"""

    prefix: str = Field(description="问题描述和解决方案")
    imports: str = Field(description="代码块导入语句")
    code: str = Field(description="不包含导入语句的代码块")


# LLM 设置
llm = ChatAnthropic(
    model="claude-3-opus-20240229",
    default_headers={"anthropic-beta": "tools-2024-04-04"},
)

# 结构化输出，include_raw=True 将捕获原始输出和解析错误
structured_llm = llm.with_structured_output(code, include_raw=True)
code_output = structured_llm.invoke(
    "编写一个打印'hello world'的 Python 程序，并用一句话解释它是如何工作的"
)

In [ ]:
# 初始推理阶段
code_output["raw"].content[0]

{'text': "<thinking>\nThe tool 'code' is relevant for writing a Python program to print a string.\n\nTo use the 'code' tool, I need values for these required parameters:\nprefix: A description of the problem and approach. I can provide this based on the request.\nimports: The import statements needed for the code. For this simple program, no imports are needed, so I can leave this blank.\ncode: The actual Python code, not including imports. I can write a simple print statement to output the string.\n\nI have all the required parameters, so I can proceed with calling the 'code' tool.\n</thinking>",
 'type': 'text'}

In [ ]:
# 工具调用
code_output["raw"].content[1]

{'text': None,
 'type': 'tool_use',
 'id': 'toolu_01UwZVQub6vL36wiBww6CU7a',
 'name': 'code',
 'input': {'prefix': "To print the string 'hello world' in Python:",
  'imports': '',
  'code': "print('hello world')"}}

In [ ]:
# JSON 字符串
code_output["raw"].content[1]["input"]

{'prefix': "To print the string 'hello world' in Python:",
 'imports': '',
 'code': "print('hello world')"}

In [ ]:
# 错误
error = code_output["parsing_error"]
error

In [ ]:
# 结果
parsed_result = code_output["parsed"]

In [7]:
parsed_result.prefix

"To print the string 'hello world' in Python:"

In [8]:
parsed_result.imports

''

In [9]:
parsed_result.code

"print('hello world')"

## 更具挑战性的示例

工具使用/结构化输出的示例用例。

这里有一些我们想要回答代码问题的文档。

In [ ]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# LCEL 文档
url = "https://python.langchain.com/docs/expression_language/"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# 根据 URL 对列表进行排序并获取文本
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)

问题：

`如果我们想要强制使用工具怎么办？`

我们可以使用回退机制。

让我们选择一个代码生成提示 - 根据我的一些测试 - 它没有正确调用工具。

我们可以看看是否能够从这里纠正。

In [ ]:
# 这个代码生成提示会调用工具
code_gen_prompt_working = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """<instructions> 你是一个在 LCEL（LangChain 表达式语言）方面有专业知识的编程助手。\n 
    这里是 LCEL 文档：\n ------- \n  {context} \n ------- \n 根据上面提供的文档回答用户问题。\n 
    确保你提供的代码可以执行，包含所有必需的导入和变量定义。\n
    按以下结构组织你的答案：1) 描述代码解决方案的前缀，2) 导入语句，3) 可运行的代码块。\n
    调用代码工具来正确地构造输出。</instructions> \n 这是用户的问题：""",
        ),
        ("placeholder", "{messages}"),
    ]
)

# 这个代码生成提示不会调用工具
code_gen_prompt_bad = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """你是一个在 LCEL（LangChain 表达式语言）方面有专业知识的编程助手。\n 
    这里是完整的 LCEL 文档：\n ------- \n  {context} \n ------- \n 根据上面提供的文档回答
    用户问题。确保你提供的代码可以执行，\n 包含所有必需的导入和变量定义。按照代码解决方案描述的结构组织你的答案。\n
    然后列出导入语句。最后列出可运行的代码块。这是用户的问题：""",
        ),
        ("placeholder", "{messages}"),
    ]
)


# 数据模型
class code(BaseModel):
    """代码输出"""

    prefix: str = Field(description="问题描述和解决方案")
    imports: str = Field(description="代码块导入语句")
    code: str = Field(description="不包含导入语句的代码块")
    description = "用于 LCEL 相关问题的代码解决方案的模式。"


# LLM 设置
llm = ChatAnthropic(
    model="claude-3-opus-20240229",
    default_headers={"anthropic-beta": "tools-2024-04-04"},
)

# 结构化输出
# include_raw=True 将捕获原始输出和解析错误
structured_llm = llm.with_structured_output(code, include_raw=True)


# 检查错误
def check_claude_output(tool_output):
    """检查解析错误或工具调用失败"""

    # 解析错误
    if tool_output["parsing_error"]:
        # 报告输出和解析错误
        print("解析错误！")
        raw_output = str(code_output["raw"].content)
        error = tool_output["parsing_error"]
        raise ValueError(
            f"解析输出时出错！请确保调用工具。输出：{raw_output}。\n 解析错误：{error}"
        )

    # 未调用工具
    elif not tool_output["parsed"]:
        print("未调用工具！")
        raise ValueError(
            "你没有使用提供的工具！请确保调用工具来构造输出。"
        )
    return tool_output


# 带输出检查的链
code_chain = code_gen_prompt_bad | structured_llm | check_claude_output

让我们添加一个检查并重试。

In [ ]:
def insert_errors(inputs):
    """在消息中插入错误"""

    # 获取错误
    error = inputs["error"]
    messages = inputs["messages"]
    messages += [
        (
            "user",
            f"重试。你需要修复解析错误：{error} \n\n 你必须调用提供的工具。",
        )
    ]
    return {
        "messages": messages,
        "context": inputs["context"],
    }


# 这将作为回退链运行
fallback_chain = insert_errors | code_chain
N = 3  # 最大重试次数
code_chain_re_try = code_chain.with_fallbacks(
    fallbacks=[fallback_chain] * N, exception_key="error"
)

In [ ]:
# 测试
messages = [("user", "如何用 LCEL 构建一个 RAG 链？")]
code_output_lcel = code_chain_re_try.invoke(
    {"context": concatenated_content, "messages": messages}
)

Failed to invoke tool!


In [15]:
parsed_result_lcel = code_output_lcel["parsed"]

In [16]:
parsed_result_lcel.prefix

"To build a RAG chain using LCEL, we'll use a vector store to retrieve relevant documents, a prompt template that incorporates the retrieved context, a chat model (like OpenAI) to generate a response based on the prompt, and an output parser to clean up the model output."

In [17]:
parsed_result_lcel.imports

'from langchain_community.vectorstores import DocArrayInMemorySearch\nfrom langchain_core.output_parsers import StrOutputParser\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom langchain_core.runnables import RunnablePassthrough\nfrom langchain_openai import ChatOpenAI, OpenAIEmbeddings'

In [18]:
parsed_result_lcel.code

'vectorstore = DocArrayInMemorySearch.from_texts(\n    ["harrison worked at kensho", "bears like to eat honey"], \n    embedding=OpenAIEmbeddings(),\n)\n\nretriever = vectorstore.as_retriever()\n\ntemplate = """Answer the question based only on the following context:\n{context}\nQuestion: {question}"""\nprompt = ChatPromptTemplate.from_template(template)\n\noutput_parser = StrOutputParser()\n\nrag_chain = (\n    {"context": retriever, "question": RunnablePassthrough()} \n    | prompt \n    | ChatOpenAI()\n    | output_parser\n)\n\nprint(rag_chain.invoke("where did harrison work?"))'

捕获错误并纠正的示例追踪：

https://smith.langchain.com/public/f06e62cb-2fac-46ae-80cd-0470b3155eae/r